# Store Traffic Monitor

Store traffic manager is an advanced use-case example, it builds on the two previous examples illustrated and shows how a user can deal with multiple videos and perform object detection and inference on all of these simultaneously.In this use-case, the application monitors the activity of people inside and outside an imaginary store front as well as keeps track of product inventory using a pre-trained neural network for detection. This application is capable of detecting objects on any number of screens.

This demo assumes you have already tried the object detector sample. If you have not done so already, we recommend going through that sample first to understand the workflow for developing a deep-learning example using OpenVINO. 

More information on using the traffic monitor, as well as the original source code can be found on github:

https://github.com/intel-iot-devkit/store-traffic-monitor-python

Note that the implementation here has been slightly modified to work on IoT DevCloud.

## Overview of How it works?

The counter uses the Inference Engine included in the OpenVINO™ toolkit. A trained neural network detects objects within a designated area by displaying a green bounding box over them. This reference implementation identifies multiple intruding objects entering the frame and identifies their class, count, and time entered. 

At start-up the sample application reads the equivalent of command line arguments and loads a network and image from the video input to the Inference Engine (IE) plugin. A job is submitted to the hardware accelerator (Intel® Core CPU, Intel® HD Graphics GPU, Intel® Xeon CPU, Intel® Movidius™ and/or Neural Compute Stick). After the inference is completed, the output videos are appropriately stored in the /results directory which can then be viewed within the Jupyter Notebook instance

## Demonstration objectives
* Video as input is supported using **OpenCV**
* Inference performed actual Edge hardware
* **OpenCV** provides the bounding boxes, labels and other information
* Visualization of the resulting bounding boxes
* Demonstrate the Async API in action

## Step 0: Setup

Import dependencies for the notebook by running the following cell

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))
from demoTools.demoutils import *

## Step 1: Advanced OpenVINO

The complete listing of source code for this example is in store_traffic_monitor.py.
This application can take multiple videos as inputs. The application is designed to work in both sync and async mode. In sync mode, the processing is done by the device by taking one frame at a time, whereas in async mode, the device takes multiple frames and processes them in parallel.


**Configure the Application**

All the configurations are written to $HOME/Reference-samples/store-traffic-monitor-python/conf_fp*.txt. We have two configuration files namely **conf_fp16.txt** and **conf_fp32.txt**

- **conf_fp16.txt**: If you want the inferences to be run on Intel® Movidius™ Neural Compute Stick as it offers native FP16 precision.
- **conf_fp32.txt**: If you want the inferences to be run on Intel® Core CPU, Intel® HD Graphics GPU and Intel® Xeon CPU.

1st line: path/to/model.xml -> 
This is the path to the model topology on the local system.
The model topology file is the .xml file that the model optimizer produces, containing the IR of the model's topology.

2nd line: path/to/model.bin ->
This is the path to the model weights on the local system.
The model weights file is the .bin file that the model optimizer produces, containing the IR of the model's weights.

3rd line: path/to/labels ->
This is a path to the labels file on the local system.
The labels file is a text file containing, all the classes/labels that the model can recognize, in the order that it was trained to recognize them (one class per line). All detection models work with integer labels and not string labels (e.g. for the mobilenet-ssd model, the **number 15 represents the class "person"**).
For the mobilenet-ssd model, we provide the class file labels.txt in the **"/data/reference-sample-data/store-traffic-monitor-python"** folder.

**Note** : store_traffic_monitor application is used to detect person and bottle in a frame. For the mobilenet-ssd model used in this example, the line number 5 and  line number 15 in the labels.txt file correspond to bottle and person respectively. If you are using a different model, then the items in labels.txt file would need to be adjusted appropriately.

Each of the following lines contain the path/to/video followed by the label to be detected on that video, e.g.:

**/data/reference-sample-data/store-traffic-monitor-python/people-detection.mp4** person

The path/to/video is the path, on the local system, to a video to use as input. The labels used must coincide with the labels from the labels file.

**Models to Use ?**

The application works with any object-detection model, provided it has the same input and output format of the mobilenet-ssd model.
The model can be any object detection model:

* that is provided by OpenVINO™ toolkit.
These can be found in the computer_vision_sdk/deployment_tools/intel_models folder.
* downloaded using the model downloader.
These can be found in the computer_vision_sdk/deployment_tools/model_downloader/object_detection folder.
* created by the user

For first-use, we recommend using the mobilenet-ssd model provided as a part of this application. The models can be found under the folder "/data/reference-sample-data/models/mobilenet-ssd"

**Videos to Use ?**

The application works with any input video. Sample videos for object detection are provided here.

For first-use, we recommend using the people-detection, one-by-one-person-detection, bottle-detection videos.
E.g.:

/data/reference-sample-data/store-traffic-monitor-python/people-detection.mp4 person<br>
/data/reference-sample-data/store-traffic-monitor-python/one-by-one-person-detection.mp4 person<br>
/data/reference-sample-data/store-traffic-monitor-python/bottle-detection.mp4 bottle

For the sake of simplicity, the videos are copied into /data/reference-sample-data/store-traffic-monitor-python/ folder

**Command line arguments options and how they are interpreted in the application source code**

```python
** python3 store_traffic_monitor.py -d CPU -c conf_fp32.txt -o results **
```

#### The description of the arguments used in the argument parser is the command line executable equivalent.

 *  -o  location where the output file with inference needs to be stored. (results/core or results/xeon or results/gpu or results/myriad)

 *  -d Type of Hardware Acceleration (CPU or GPU or MYRIAD)
 *  -c configuration file to use ("conf_fp32.txt" for Intel® Core CPU, Intel® HD Graphics GPU and Intel® Xeon CPU. "conf_fp16.txt" for Intel® Movidius™ Neural Compute Stick)



                        
### 1.1 Choosing Device
First, we must select the device used for the inferencing. This is done by loading the appropriate plugin to initialize the specified device (CPU, GPU, MYRIAD etc) and load the extensions library (if specified) provided in the extension/ folder for the device.

The plugin class is IEPlugin and can be constructed as follows:
```python
    # Parses the configuration file "conf.txt" and reads model_xml, model_bin, labels_file, videoCaps
    parse_conf_file()
    # Plugin initialization for specified device and load extensions library
    print("Initializing plugin for {} device...".format(TARGET_DEVICE))
    plugin = IEPlugin(device=TARGET_DEVICE)
    if 'CPU' in TARGET_DEVICE:
        plugin.add_cpu_extension(CPU_EXTENSION)
```

**Note**: Currently, three types of plugins are supported: CPU, GPU, and MYRIAD CPU plugin may require additional extensions to improve performance. Use add_cpu_extension function to load these additional extensions.

### 1.2 Read the IR (Intermediate Representation) model

Intel Model Optimizer creates Intermediate Representation (IR) models that are optimized for different Intel hardware.
We can import these optimized models (weights) into our neural network using **`IENetwork`**. 
```python
    net = IENetwork.from_ir(model=model_xml, weights=model_bin)
    assert len(net.inputs.keys()) == 1, "Sample supports only single input topologies"
    assert len(net.outputs) == 1, "Sample supports only single output topologies"
    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
```

### 1.3 Load the network on to the model

Once we have the plugin and the network, we can load the IR into the plugin using **`plugin.load`**.

```python
# Loading IR model to the plugin.
    exec_net = plugin.load(network=net, num_requests=2)
    # Read and pre-process input image
    n, c, h, w = net.inputs[input_blob]
    del net

```

### 1.4 Start video capture using OpenCV 

Now we are ready to capture the frames from the video sample using **OpenCV VideoCapture** API.
Upon getting the frame we are ready to perform inference.

```python
cap = cv2.VideoCapture(input_stream)
```


In [ ]:
!/opt/intel/computer_vision_sdk/deployment_tools/model_downloader/downloader.py --print_all

In [ ]:
!/opt/intel/computer_vision_sdk/deployment_tools/model_downloader/downloader.py --name mobilenet-ssd -o raw_models

In [ ]:
!/opt/intel/computer_vision_sdk/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP32 \
-o models/mobilenet-ssd/FP32 \
--scale 256 \
--mean_values [127,127,127] 

In [ ]:
!/opt/intel/computer_vision_sdk/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP16 \
-o models/mobilenet-ssd/FP16 \
--scale 256 \
--mean_values [127,127,127] 

## Step 2: Running on the IoT DevCloud

### 2.1 Creating job file

First, we need to create a job file.
The job file is written in bash, and will be executed on the compute node.
For this example, we have written the job file for you in the notebook.
Run the following cell to write this in to the file "store_traffic_manager_job.sh"

In [ ]:
%%writefile store_traffic_monitor_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR

# Traffic monitor script writes output to a file inside a directory. We make sure that this directory exists.
#  The output directory is the first argument of the bash script
mkdir -p $1

if [ "$2" = "HETERO:FPGA,CPU" ]; then
    source /opt/intel/computer_vision_sdk/bin/setup_hddl.sh
    aocl program acl0 /opt/intel/computer_vision_sdk_2018.4.420/bitstreams/a10_vision_design_bitstreams/4-0_PL1_FP11_MobileNet_ResNet_VGG_Clamp.aocx
fi

if [ "$3" = "FP32"]; then
    config_file="conf_fp32.txt"
else
    config_file="conf_fp16.txt"
fi

SAMPLEPATH=$PBS_O_WORKDIR
# Traffic monitor takes 3 inputs, which are passed in as arguments to the bash script. 
#  -o : output directory of the videos
#  -d : device to use for inference
#  -c : conf file to use
#  -n : number of videos to process

python3 store_traffic_monitor.py  -d $2 \
                                    -m ${SAMPLEPATH}/models/mobilenet-ssd/$3/mobilenet-ssd.xml \
                                    -l labels.txt \
                                    -e /opt/intel/computer_vision_sdk/deployment_tools/inference_engine/lib/ubuntu_16.04/intel64/libcpu_extension_avx2.so \
                                    -lp false \
                                    -o $1 \
                                    -c $config_file \
                                    -n $4

### 2.2 Submitting to Job Queue

Now that we have the job script, we can submit the jobs to compute nodes using the `qsub` command.
Let's try submitting the store_traffic_monitor job to 4 different types of nodes.

There are three options of `qsub` command that we use for this:
- `-l` : this option lets us select the number and the type of nodes using `nodes={node_count}:{property}`. 
- `-F` : this option lets us send arguments to the bash script. 
- `-N` : this option lets us name the job so that it is easier to distinguish between them.

To see the available types of nodes (e.g. properties) on the IoT DevCloud, run the following cell.

In [ ]:
!pbsnodes | grep properties | sort | uniq -c

Here, the properties describe the node, and number on the left is the number of available nodes.

Now let's actually submit the jobs. 
Each of the next 4 cells will submit jobs to different nodes.
The output of the cell is the `JobID` of your job, which you can use to track progress of a job.

#### submitting to a node with Intel® Core CPU

In [ ]:
print("Submitting job to Intel Core CPU...")
#Submit job to the queue
job_id_core =!qsub store_traffic_monitor_job.sh -l nodes=1:iei-tank-core -F "results/core CPU FP32 3" -N monitor_core

#Progress indicators
if job_id_core:
    progressIndicator('results/core', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)

#### submitting to a node with Intel® Xeon CPU

In [ ]:
print("Submitting job to Intel Xeon CPU...")
#Submit job to the queue
job_id_xeon = !qsub store_traffic_monitor_job.sh -l nodes=1:iei-tank-xeon -F "results/xeon CPU FP32 3" -N monitor_xeon
    
if job_id_xeon:
    progressIndicator('results/xeon', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)    

#### submitting to a node with Intel® Core CPU and using the onboard Intel GPU

In [ ]:
print("Submitting job to Intel Core CPU with Intel GPU...")
#Submit job to the queue
job_id_gpu = !qsub store_traffic_monitor_job.sh -l nodes=1:iei-tank-core -F "results/gpu GPU FP32 3" -N monitor_gpu
    
#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)

#### submitting to a node with Intel® Movidius Stick

In [ ]:
print("Submitting job to node with Intel Movidius NCS...")
#Submit job to the queue
job_id_myriad = !qsub store_traffic_monitor_job.sh -l nodes=1:iei-tank-movidius -F "results/myriad MYRIAD FP16 3" -N monitor_myriad
    
#Progress indicators
if job_id_myriad:
    progressIndicator('results/myriad', 'i_progress_'+job_id_myriad[0]+'.txt', "Inference", 0, 100)

#### submitting to a node with Intel FPGA HDDL-F (High Density Deep Learning)

In [ ]:
print("Submitting job to node with Intel FPGA HDDL-F...")
#Submit job to the queue
job_id_fpga = !qsub store_traffic_monitor_job.sh -l nodes=1:iei-tank-fpga -F "results/fpga HETERO:FPGA,CPU FP32 3" -N monitor_fpga
    
#Progress indicators
if job_id_fpga:
    progressIndicator('results/fpga', 'i_progress_'+job_id_fpga[0]+'.txt', "Inference", 0, 100)

### 2.3 Check if the jobs are done

Run the following cell to bring the custom qstat widget. 

In [ ]:
liveQstat()

You should see the jobs you have submitted (referenced by `Job ID`).
It should also show the jupyter notebook job as well. 

### Before moving to step 3, make sure that all the monitor_*  jobs submitted to the queue are completed.

## Step 3: Results

Once the jobs are complete, the stdout and stderr are store in files with names of the form (based on our `-N` argument):

`monitor_{type}.o{JobID}`

`monitor_{type}.e{JobID}`

But for this script, the main output is the mp4 videos which are stored in the `results/` directory.
We wrote a short utility script that will display these videos in the notebook.
See `demoutils.py` if interested in the script.

Then run the following four cells to see the output  from out jobs.

In [ ]:
videoHTML('IEI Tank (Intel® Core CPU)', ['results/core/inference_output_Video_0.mp4', 'results/core/inference_output_Video_1.mp4', 'results/core/inference_output_Video_2.mp4', 'results/core/Statistics.mp4'], 'results/core/stats.txt')

In [ ]:
videoHTML('IEI Tank Xeon (Intel® Xeon CPU)',['results/xeon/inference_output_Video_0.mp4', 'results/xeon/inference_output_Video_1.mp4', 'results/xeon/inference_output_Video_2.mp4', 'results/xeon/Statistics.mp4'], 'results/xeon/stats.txt')

In [ ]:
videoHTML('IEI Intel GPU (Intel® Onboard GPU)', ['results/gpu/inference_output_Video_0.mp4', 'results/gpu/inference_output_Video_1.mp4', 'results/gpu/inference_output_Video_2.mp4', 'results/gpu/Statistics.mp4'], 'results/gpu/stats.txt')

In [ ]:
videoHTML('IEI Tank + Myriad (Intel® Core + Movidius)',['results/myriad/inference_output_Video_0.mp4', 'results/myriad/inference_output_Video_1.mp4', 'results/myriad/inference_output_Video_2.mp4', 'results/myriad/Statistics.mp4'], 'results/myriad/stats.txt')

In [ ]:
videoHTML('IEI Tank + Intel FPGA HDDL-F',['results/fpga/inference_output_Video_0.mp4', 'results/fpga/inference_output_Video_1.mp4', 'results/fpga/inference_output_Video_2.mp4', 'results/fpga/Statistics.mp4'], 'results/fpga/stats.txt')

In [ ]:
summaryPlot({'results/core/stats.txt':'Core', 'results/xeon/stats.txt':'Xeon', 'results/gpu/stats.txt':'GPU', 'results/fpga/stats.txt':'FPGA', 'results/myriad/stats.txt':'Myriad'}, 'Architecture', 'Time, seconds', 'Inference engine processing time' )